## XGBSurv Efron Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.efron_final import efron_likelihood, get_cumulative_hazard_function_efron
from pycox.evaluation import EvalSurv
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
from sklearn.utils.fixes import loguniform

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50 #0
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
model = 'xgbsurv_efron_'

# set seed for scipy
np.random.seed(rand_state)

# Define parameter grid for random forest classifier
param_grid = {
'estimator__reg_alpha': scloguniform(1e-10,1),#[1e-10,1], # from hyp augmentation, L1 regularization
'estimator__reg_lambda': scloguniform(1e-10,1), #[1e-10,1], #alias l2_regularization, lambda in augmentation
'estimator__learning_rate': scloguniform(0.01,1), #[0.001,1], # assumed alias eta from augmentation,
'estimator__n_estimators':  scrandint(1,4000), # corresponds to num_rounds
'estimator__gamma': loguniform(0.001, 1.0),#[0.1,1], # minimum loss reduction required to make a further partition on a leaf node of the tree.
'estimator__colsample_bylevel': scuniform(0.1, 1-0.1), #[0.1,1], # from hyp augmentation
'estimator__colsample_bynode': scuniform(0.1, 1-0.1), #[0.1,1], # from hyp augmentation, uniform(0.1,1),
'estimator__colsample_bytree': scuniform(0.5, 1-0.5),#[0.5,1], # from hyp augmentation, seems to exceed the bound, uniform(0.5,1)
'estimator__max_depth': scrandint(1,20),#[1,20], # from hyp augmentation
'estimator__max_delta_step': scrandint(0,10),#[0,10], # from hyp augmentation
'estimator__min_child_weight' : scloguniform(0.1,20-0.1),#[0.1,20], # from hyp augmentation
'estimator__subsample': scuniform(0.01,1-0.01),#[0.01,1], # from hyp augmentation
}

## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)
inner_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)

## Scoring Function

In [4]:
scoring_function = make_scorer(efron_likelihood, greater_is_better=False)

## Set Model & Train Test Evaluate

In [5]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] #, load_flchain, load_rgbsg, load_support, load_tcga]
data_set_fns_str = ['load_metabric', 'load_flchain', 'load_rgbsg', 'load_support'] 
one_hot_dict = {'load_flchain': ['mgus'],  'load_rgbsg':['grade'],'load_support':['cancer', 'race']}
agg_metrics_cindex = []
agg_metrics_ibs = []

for idx, dataset in enumerate(data_set_fns):
    model = 'xgbsurv_efron_'
    # get name of current dataset
    data = dataset(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(data_set_fns_str[idx])
    if data_set_fns_str[idx] in one_hot_dict.keys():
       X = pd.get_dummies(X, columns=one_hot_dict[data_set_fns_str[idx]])
    print(X.dtypes)
    X, y = sort_X_y_pandas(X, y)
    #print(X.dtypes)
    dataset_name = filename.split('_')[0]
    # add IBS later
    outer_scores = {'cindex_train_'+dataset_name:[], 'cindex_test_'+dataset_name:[],
                    'ibs_train_'+dataset_name:[], 'ibs_test_'+dataset_name:[]}
    best_params = {'best_params_'+dataset_name:[]}
    best_model = {'best_model_'+dataset_name:[]}
    ct = make_column_transformer(
            (StandardScaler(), make_column_selector(dtype_include=['float32'])),
            #(OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist'), make_column_selector(dtype_include=['category', 'object'])),
            remainder='passthrough')
    
    estimator = XGBSurv(
        objective='efron_objective',
        eval_metric='efron_loss',
        random_state=rand_state, 
        disable_default_eval_metric=True,
        early_stopping_rounds=early_stopping_rounds, 
        base_score=base_score,
                        )
    pipe = Pipeline([('scaler',ct),
                    ('estimator', estimator)])
    
    rs = RandomizedSearchCV(pipe, param_grid, scoring = scoring_function, n_jobs=-1, 
                             cv=inner_custom_cv, n_iter=n_iter, refit=True, random_state=rand_state)
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        # Split data into training and testing sets for outer fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train, y_train = sort_X_y_pandas(X_train, y_train)
        X_test, y_test = sort_X_y_pandas(X_test, y_test)

        #print(X_train.shape, type(X_train))
        #print(y_train.shape, type(y_train))
        #print(y_test.shape, type(y_test))
        #print(X_test.shape, type(X_test))
        # save splits and data
        np.savetxt('splits/'+model+'train_index_'+str(i)+'_'+filename, train_index, delimiter=',')
        np.savetxt('splits/'+model+'test_index_'+str(i)+'_'+filename, test_index, delimiter=',')
        
        #np.savetxt('splits/'+model+'X_train_'+str(i)+'_'+filename, X_train, delimiter=',')
        #np.savetxt('splits/'+model+'X_test_'+str(i)+'_'+filename, X_test, delimiter=',')

        #np.savetxt('splits/'+model+'y_train_'+str(i)+'_'+filename, y_train, delimiter=',')
        #np.savetxt('splits/'+model+'y_test_'+str(i)+'_'+filename, y_test, delimiter=',')

        rs.fit(X_train, y_train, estimator__eval_test_size=validation_size, estimator__verbose=0)
        best_preds_train = rs.best_estimator_.predict(X_train)
        best_preds_test = rs.best_estimator_.predict(X_test)
        np.savetxt('predictions/'+model+'best_preds_train_'+str(i)+'_'+filename, best_preds_train, delimiter=',')
        np.savetxt('predictions/'+model+'best_preds_test_'+str(i)+'_'+filename, best_preds_test, delimiter=',')


        # save hyperparameter settings
        params = rs.best_estimator_.get_params
        best_params['best_params_'+dataset_name] += [rs.best_params_]
        best_model['best_model_'+dataset_name] += [params]
        try:
            cum_hazard_train = get_cumulative_hazard_function_efron(
                    X_train.values, X_train.values, y_train.values, y_train.values,
                    best_preds_train.reshape(-1), best_preds_train.reshape(-1)
                    )

            df_survival_train = np.exp(-cum_hazard_train)
            durations_train, events_train = transform_back(y_train.values)
            time_grid_train = np.linspace(durations_train.min(), durations_train.max(), 100)
            ev = EvalSurv(df_survival_train, durations_train, events_train, censor_surv='km')
            print('Concordance Index',ev.concordance_td('antolini'))
            print('Integrated Brier Score:',ev.integrated_brier_score(time_grid_train))
            cindex_score_train = ev.concordance_td('antolini')
            ibs_score_train = ev.integrated_brier_score(time_grid_train)

            outer_scores['cindex_train_'+dataset_name] += [cindex_score_train]
            outer_scores['ibs_train_'+dataset_name] += [ibs_score_train]

        except:
            outer_scores['cindex_train_'+dataset_name] += [np.nan]
            outer_scores['ibs_train_'+dataset_name] += [np.nan]
            
        try:
            cum_hazard_test = get_cumulative_hazard_function_efron(
                    X_train.values, X_test.values, y_train.values, y_test.values,
                    best_preds_train.reshape(-1), best_preds_test.reshape(-1)
                    )
            df_survival_test = np.exp(-cum_hazard_test)
            durations_test, events_test = transform_back(y_test.values)
            print('durations',durations_test.min(), durations_test.max())
            time_grid_test = np.linspace(durations_test.min(), durations_test.max(), 100)
            ev = EvalSurv(df_survival_test, durations_test, events_test, censor_surv='km')
            print('Concordance Index',ev.concordance_td('antolini'))
            print('Integrated Brier Score:',ev.integrated_brier_score(time_grid_test))
            cindex_score_test = ev.concordance_td('antolini')
            ibs_score_test = ev.integrated_brier_score(time_grid_test)

            outer_scores['cindex_test_'+dataset_name] += [cindex_score_test]
            outer_scores['ibs_test_'+dataset_name] += [ibs_score_test]
        except: 
            outer_scores['cindex_test_'+dataset_name] += [np.nan]
            outer_scores['ibs_test_'+dataset_name] += [np.nan]
            
    df_best_params = pd.DataFrame(best_params)
    df_best_model = pd.DataFrame(best_model)
    df_outer_scores = pd.DataFrame(outer_scores)
    df_metrics = pd.concat([df_best_params,df_best_model,df_outer_scores], axis=1)
    df_metrics.to_csv('metrics/'+model+'metric_summary_'+filename, index=False)
    # cindex
    df_agg_metrics_cindex = pd.DataFrame({'dataset':[dataset_name],
                                            'cindex_train_mean':df_outer_scores['cindex_train_'+dataset_name].mean(),
                                            'cindex_train_std':df_outer_scores['cindex_train_'+dataset_name].std(),
                                            'cindex_test_mean':df_outer_scores['cindex_test_'+dataset_name].mean(),
                                            'cindex_test_std':df_outer_scores['cindex_test_'+dataset_name].std() })
    # IBS
    df_agg_metrics_ibs = pd.DataFrame({'dataset':[dataset_name],
                                            'ibs_train_mean':df_outer_scores['ibs_train_'+dataset_name].mean(),
                                            'ibs_train_std':df_outer_scores['ibs_train_'+dataset_name].std(),
                                            'ibs_test_mean':df_outer_scores['ibs_test_'+dataset_name].mean(),
                                            'ibs_test_std':df_outer_scores['ibs_test_'+dataset_name].std() })
    agg_metrics_cindex.append(df_agg_metrics_cindex)
    agg_metrics_ibs.append(df_agg_metrics_ibs)

df_final_breslow_1_cindex = pd.concat([df for df in agg_metrics_cindex]).round(4)
df_final_breslow_1_cindex.to_csv('metrics/final_gbdt_efron_1_cindex.csv', index=False)
df_final_breslow_1_cindex.to_csv('/Users/JUSC/Documents/644928e0fb7e147893e8ec15/05_thesis/tables/final_gbdt_efron_1_cindex.csv', index=False)  #


df_final_breslow_1_ibs = pd.concat([df for df in agg_metrics_ibs]).round(4)
df_final_breslow_1_ibs.to_csv('metrics/final_gbdt_efron_1_ibs.csv', index=False)
df_final_breslow_1_ibs.to_csv('/Users/JUSC/Documents/644928e0fb7e147893e8ec15/05_thesis/tables/final_gbdt_efron_1_ibs.csv', index=False) 



load_metabric
MKI67                float32
EGFR                 float32
PGR                  float32
ERBB2                float32
hormone_treatment      uint8
radiotherapy           uint8
chemotherapy           uint8
ER_positive            uint8
age                  float32
dtype: object


KeyboardInterrupt: 

## TCGA Train, Test, Evaluation

In [6]:
param_grid = {
'estimator__reg_alpha': scloguniform(1e-10,1),#[1e-10,1], # from hyp augmentation, L1 regularization
'estimator__reg_lambda': scloguniform(1e-10,1), #[1e-10,1], #alias l2_regularization, lambda in augmentation
'estimator__learning_rate': scloguniform(0.001,1), #[0.001,1], # assumed alias eta from augmentation,
'estimator__n_estimators':  scrandint(1,4000), # corresponds to num_rounds
'estimator__gamma': loguniform(0.001,1.0),#[0.1,1], # minimum loss reduction required to make a further partition on a leaf node of the tree.
'estimator__colsample_bylevel': scuniform(0.1, 1-0.1), #[0.1,1], # from hyp augmentation
'estimator__colsample_bynode': scuniform(0.1, 1-0.1), #[0.1,1], # from hyp augmentation, uniform(0.1,1),
'estimator__colsample_bytree': scuniform(0.5, 1-0.5),#[0.5,1], # from hyp augmentation, seems to exceed the bound, uniform(0.5,1)
'estimator__max_depth': scrandint(1,20),#[1,20], # from hyp augmentation
'estimator__max_delta_step': scrandint(0,10),#[0,10], # from hyp augmentation
'estimator__min_child_weight' : scloguniform(0.1,20-0.1),#[0.1,20], # from hyp augmentation
'estimator__subsample': scuniform(0.01,1-0.01),#[0.01,1], # from hyp augmentation
#'pca__n_components': [8, 16, 32, 64]
}

In [9]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD']
agg_metrics_cindex = []
agg_metrics_ibs = []


for idx, cancer_type in enumerate(cancer_types):
    model = 'xgbsurv_efron_'
    # get name of current dataset
    data = load_tcga(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", cancer_type=cancer_type, as_frame=True)
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()
    filename = data.filename
    dataset_name = filename.split('_')[0]
    X, y = sort_X_y_pandas(X, y)

    # add IBS later
    outer_scores = {'cindex_train_'+dataset_name:[], 'cindex_test_'+dataset_name:[],
                    'ibs_train_'+dataset_name:[], 'ibs_test_'+dataset_name:[]}
    best_params = {'best_params_'+dataset_name:[]}
    best_model = {'best_model_'+dataset_name:[]}
    ct = make_column_transformer(
            (StandardScaler(), make_column_selector(dtype_include=['float32']))
            ,remainder='passthrough')
    
    estimator = XGBSurv(
        objective='efron_objective',
        eval_metric='efron_loss',
        disable_default_eval_metric=True,
        early_stopping_rounds=early_stopping_rounds, 
        base_score=base_score,
        random_state=rand_state
                        )
    
    pipe = Pipeline([('scaler',ct),
                    #('pca', PCA()),
                    ('estimator', estimator)])
    
    rs = RandomizedSearchCV(pipe, param_grid, scoring = scoring_function, n_jobs=-1, 
                             cv=inner_custom_cv, n_iter=n_iter, refit=True, random_state=rand_state)
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        # Split data into training and testing sets for outer fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train, y_train = sort_X_y_pandas(X_train, y_train)
        X_test, y_test = sort_X_y_pandas(X_test, y_test)

        #print(y_train.shape, type(y_train))
        #print(X_train.shape, type(X_train))
        #print(X_test.shape, type(X_test))
        #print(y_test.shape, type(y_test))
        # save splits and data
        np.savetxt('splits/'+model+'train_index_'+str(i)+'_'+filename, train_index, delimiter=',')
        np.savetxt('splits/'+model+'test_index_'+str(i)+'_'+filename, test_index, delimiter=',')
        
        #np.savetxt('splits/'+model+'X_train_'+str(i)+'_'+filename, X_train, delimiter=',')
        #np.savetxt('splits/'+model+'X_test_'+str(i)+'_'+filename, X_test, delimiter=',')

        #np.savetxt('splits/'+model+'y_train_'+str(i)+'_'+filename, y_train, delimiter=',')
        #np.savetxt('splits/'+model+'y_test_'+str(i)+'_'+filename, y_test, delimiter=',')

        rs.fit(X_train, y_train, estimator__eval_test_size=0.2, estimator__verbose=0)
        best_preds_train = rs.best_estimator_.predict(X_train)
        best_preds_test = rs.best_estimator_.predict(X_test)
        np.savetxt('predictions/'+model+'best_preds_train_'+str(i)+'_'+filename, best_preds_train, delimiter=',')
        np.savetxt('predictions/'+model+'best_preds_test_'+str(i)+'_'+filename, best_preds_test, delimiter=',')

        # save hyperparameter settings
        params = rs.best_estimator_.get_params
        best_params['best_params_'+dataset_name] += [rs.best_params_]
        best_model['best_model_'+dataset_name] += [params]
        try:
            cum_hazard_train = get_cumulative_hazard_function_efron(
                    X_train.values, X_train.values, y_train.values, y_train.values,
                    best_preds_train.reshape(-1), best_preds_train.reshape(-1)
                    )

            df_survival_train = np.exp(-cum_hazard_train)
            durations_train, events_train = transform_back(y_train.values)
            time_grid_train = np.linspace(durations_train.min(), durations_train.max(), 100)
            ev = EvalSurv(df_survival_train, durations_train, events_train, censor_surv='km')
            print('Concordance Index',ev.concordance_td('antolini'))
            print('Integrated Brier Score:',ev.integrated_brier_score(time_grid_train))
            cindex_score_train = ev.concordance_td('antolini')
            ibs_score_train = ev.integrated_brier_score(time_grid_train)

            outer_scores['cindex_train_'+dataset_name] += [cindex_score_train]
            outer_scores['ibs_train_'+dataset_name] += [ibs_score_train]

        except:
            outer_scores['cindex_train_'+dataset_name] += [np.nan]
            outer_scores['ibs_train_'+dataset_name] += [np.nan]
            
        try:
            cum_hazard_test = get_cumulative_hazard_function_efron(
                    X_train.values, X_test.values, y_train.values, y_test.values,
                    best_preds_train.reshape(-1), best_preds_test.reshape(-1)
                    )
            df_survival_test = np.exp(-cum_hazard_test)
            durations_test, events_test = transform_back(y_test.values)
            print('durations',durations_test.min(), durations_test.max())
            time_grid_test = np.linspace(durations_test.min(), durations_test.max(), 100)
            ev = EvalSurv(df_survival_test, durations_test, events_test, censor_surv='km')
            print('Concordance Index',ev.concordance_td('antolini'))
            print('Integrated Brier Score:',ev.integrated_brier_score(time_grid_test))
            cindex_score_test = ev.concordance_td('antolini')
            ibs_score_test = ev.integrated_brier_score(time_grid_test)

            outer_scores['cindex_test_'+dataset_name] += [cindex_score_test]
            outer_scores['ibs_test_'+dataset_name] += [ibs_score_test]
        except: 
            outer_scores['cindex_test_'+dataset_name] += [np.nan]
            outer_scores['ibs_test_'+dataset_name] += [np.nan]
            
    df_best_params = pd.DataFrame(best_params)
    df_best_model = pd.DataFrame(best_model)
    df_outer_scores = pd.DataFrame(outer_scores)
    df_metrics = pd.concat([df_best_params,df_best_model,df_outer_scores], axis=1)
    df_metrics.to_csv('metrics/'+model+'metric_summary_'+filename, index=False)
    
    # cindex
    df_agg_metrics_cindex = pd.DataFrame({'dataset':[dataset_name],
                                            'cindex_train_mean':df_outer_scores['cindex_train_'+dataset_name].mean(),
                                            'cindex_train_std':df_outer_scores['cindex_train_'+dataset_name].std(),
                                            'cindex_test_mean':df_outer_scores['cindex_test_'+dataset_name].mean(),
                                            'cindex_test_std':df_outer_scores['cindex_test_'+dataset_name].std() })
    # IBS
    df_agg_metrics_ibs = pd.DataFrame({'dataset':[dataset_name],
                                            'ibs_train_mean':df_outer_scores['ibs_train_'+dataset_name].mean(),
                                            'ibs_train_std':df_outer_scores['ibs_train_'+dataset_name].std(),
                                            'ibs_test_mean':df_outer_scores['ibs_test_'+dataset_name].mean(),
                                            'ibs_test_std':df_outer_scores['ibs_test_'+dataset_name].std() })

    agg_metrics_cindex.append(df_agg_metrics_cindex)
    agg_metrics_ibs.append(df_agg_metrics_ibs)


    

Concordance Index 0.5839311334289814
Integrated Brier Score: 0.20449029783416708
durations 6.0 3308.0
Concordance Index 0.5498426023084995
Integrated Brier Score: 0.21370030080381322
Concordance Index 0.12550448430493275
Integrated Brier Score: 0.2133100706775016
durations 1.0 3437.0
Concordance Index 0.10789980732177264
Integrated Brier Score: 0.2018971424011434
Concordance Index 0.5913586673607496
Integrated Brier Score: 0.20047267658568269
durations 8.0 3675.0
Concordance Index 0.5792163543441227
Integrated Brier Score: 0.23261824266465217
Concordance Index 0.6210485089240754
Integrated Brier Score: 0.20583885121964862
durations 6.0 3478.0
Concordance Index 0.6469465648854962
Integrated Brier Score: 0.20902797890493874
Concordance Index 0.590408943807002
Integrated Brier Score: 0.1987154484787017
durations 6.0 2728.0
Concordance Index 0.3982011447260834
Integrated Brier Score: 0.2294498975809528
Concordance Index 0.5992082495291
Integrated Brier Score: 0.1948113117771023
durations 4

In [14]:
# for name in ['BLCA', 'BRCA', 'HNSC', 'KIRC','LGG']:
#     df_outer_scores = pd.read_csv('metrics/xgbsurv_efron_metric_summary_'+name+'_adapted.csv')
#     dataset_name = name #+'_adapted'
#     # cindex
#     df_agg_metrics_cindex = pd.DataFrame({'dataset':[dataset_name],
#                                             'cindex_train_mean':df_outer_scores['cindex_train_'+dataset_name].mean(),
#                                             'cindex_train_std':df_outer_scores['cindex_train_'+dataset_name].std(),
#                                             'cindex_test_mean':df_outer_scores['cindex_test_'+dataset_name].mean(),
#                                             'cindex_test_std':df_outer_scores['cindex_test_'+dataset_name].std() })
#     # IBS
#     df_agg_metrics_ibs = pd.DataFrame({'dataset':[dataset_name],
#                                             'ibs_train_mean':df_outer_scores['ibs_train_'+dataset_name].mean(),
#                                             'ibs_train_std':df_outer_scores['ibs_train_'+dataset_name].std(),
#                                             'ibs_test_mean':df_outer_scores['ibs_test_'+dataset_name].mean(),
#                                             'ibs_test_std':df_outer_scores['ibs_test_'+dataset_name].std() })
    
#     agg_metrics_cindex.append(df_agg_metrics_cindex)
#     agg_metrics_ibs.append(df_agg_metrics_ibs)

In [19]:
df_final_efron_1_cindex = pd.concat([df for df in agg_metrics_cindex]).round(4)
df_final_efron_1_cindex.sort_values('dataset', ascending=True, inplace=True)
df_final_efron_1_cindex.to_csv('metrics/final_gbdt_tcga_efron_1_cindex.csv', index=False)
df_final_efron_1_cindex.to_csv('/Users/JUSC/Documents/644928e0fb7e147893e8ec15/05_thesis/tables/final_gbdt_tcga_efron_1_cindex.csv', index=False)  #
df_final_efron_1_cindex 

,dataset,cindex_train_mean,cindex_train_std,cindex_test_mean,cindex_test_std
0,BLCA,0.5077,0.0944,0.5042,0.1016
0,BRCA,0.5047,0.0838,0.4832,0.1567
0,HNSC,0.5115,0.1449,0.4749,0.1250
0,KIRC,0.5897,0.0827,0.5715,0.0878
0,LGG,0.5906,0.1828,0.5932,0.1658
0,LIHC,0.5025,0.2112,0.4564,0.2150
0,LUAD,0.5341,0.0705,0.5044,0.0660
0,LUSC,0.3931,0.1400,0.4056,0.1121
0,OV,0.4686,0.0574,0.4170,0.1009
0,STAD,0.3698,0.1180,0.3853,0.1730


In [21]:
df_final_efron_1_ibs = pd.concat([df for df in agg_metrics_ibs]).round(4)
df_final_efron_1_ibs.sort_values('dataset', ascending=True, inplace=True)
df_final_efron_1_ibs.to_csv('metrics/final_gbdt_tcga_efron_1_ibs.csv', index=False)
df_final_efron_1_ibs.to_csv('/Users/JUSC/Documents/644928e0fb7e147893e8ec15/05_thesis/tables/final_gbdt_tcga_efron_1_ibs.csv', index=False) 
df_final_efron_1_ibs

,dataset,ibs_train_mean,ibs_train_std,ibs_test_mean,ibs_test_std
0,BLCA,0.2142,0.0043,0.2225,0.0059
0,BRCA,0.1905,0.0047,0.1960,0.0092
0,HNSC,0.1834,0.0056,0.2039,0.0072
0,KIRC,0.2046,0.0037,0.2021,0.0075
0,LGG,0.1727,0.0055,0.1931,0.0113
0,LIHC,0.2046,0.0057,0.2173,0.0132
0,LUAD,0.1896,0.0050,0.1985,0.0114
0,LUSC,0.1913,0.0051,0.2038,0.0120
0,OV,0.1272,0.0077,0.1440,0.0182
0,STAD,0.2085,0.0060,0.2170,0.0082


In [13]:
outer_scores

{'cindex_train_STAD': [0.5500381647018913,
  0.289766081871345,
  0.2765848013372766,
  0.30280492987675306,
  0.4297555973337891],
 'cindex_test_STAD': [0.62217659137577,
  0.30457227138643067,
  0.1570957095709571,
  0.38636363636363635,
  0.4562913907284768],
 'ibs_train_STAD': [0.2054527141238668,
  0.2060189217552257,
  0.2094701367953498,
  0.21841435382217075,
  0.20297241084123877],
 'ibs_test_STAD': [0.2132553794866624,
  0.2151660488621382,
  0.21547506636800412,
  0.20994497484322663,
  0.23122326027492726]}